### Data Extraction using Spotify API - in real time

#### Please note that this is real-time data, retrieved via the API, will be updated dynamically and can change based on the time of execution (September)

To access spotify library

pip install spotipy

To access data from spotify & for authorization

In [ ]:
import spotipy       
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd

paste client id & secret from spotify developer account (app) $ also for authentication

In [ ]:
client_credentials_manager = SpotifyClientCredentials(client_id = "",client_secret = "")

In [ ]:
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

playlist_link of top 50 songs globally trending 

In [ ]:
playlist_link = "https://open.spotify.com/playlist/37i9dQZEVXbNG2KDcFcKOF?si=1333723a6eff4b7f"

In [ ]:
playlist_uri = playlist_link.split("/")[-1].split("?")[0]

In [ ]:
print(playlist_uri)

In [ ]:
data = sp.playlist_tracks(playlist_uri)

In [ ]:
data['items']   #contain album , artist and song details

In [ ]:
len(data['items'])

In [ ]:
data['items'][0]['track']['album']

In [ ]:
data['items'][0]['track']['album']['id']

In [ ]:
data['items'][0]['track']['album']['name']

In [ ]:
data['items'][0]['track']['album']['release_date']

In [ ]:
data['items'][0]['track']['album']['total_tracks']

In [ ]:
data['items'][0]['track']['album']['external_urls']

In [ ]:
data['items'][0]['track']['album']['external_urls']['spotify']

In [ ]:
album_list = []
for row in data['items']:
    album_info = row['track']['album']
    
    album_id = album_info['id']
    album_name = album_info['name']
    album_release_date = album_info['release_date']
    album_total_tracks = album_info['total_tracks']
    album_url = album_info['external_urls']['spotify']
    album_cover_art_url = album_info['images'][0]['url'] if album_info['images'] else None  
    length = f"{row['track']['duration_ms'] // 60000}:{(row['track']['duration_ms'] // 1000) % 60:02}"


    album_element = {
        'album_id': album_id,
        'album_name': album_name,
        'release_date': album_release_date,
        'total_tracks': album_total_tracks,
        'url': album_url,
        'cover_art': album_cover_art_url, 
        'length': length, 
        'song_ids': [row['track']['id']] 
    }
    
    # Check if album already exists to avoid duplicates
    if album_element not in album_list:
        album_list.append(album_element)


In [ ]:
album_list

In [ ]:
data['items'][1]['track']['artists']

In [ ]:
artist_list = []
for row in data['items']:
    for key, value in row.items():
        if key == 'track':
            song_id = value['id']  
            track_number = value['track_number']  
            
            for artist in value['artists']:
                artist_dict = {
                    'artist_id': artist['id'],
                    'artist_name': artist['name'],
                    'external_url': artist['href'],
                    'song_id': song_id, 
                    'track_number': track_number  
                }
                artist_list.append(artist_dict)


In [ ]:
artist_list

In [ ]:
song_list = []
for row in data['items']:
    song_id = row['track']['id']
    song_name = row['track']['name']
    song_duration = row['track']['duration_ms']
    song_url = row['track']['external_urls']['spotify']
    song_popularity = row['track']['popularity']
    song_added = row['added_at']
    explicit_flag = row['track']['explicit']  

    
    song_element = {
        'song_id': song_id,
        'song_name': song_name,
        'duration_ms': song_duration,
        'url': song_url,
        'popularity': song_popularity,
        'song_added': song_added,
        'explicit': explicit_flag,
    }
    
    # Append the song element to the list
    song_list.append(song_element)


In [ ]:
album_df = pd.DataFrame.from_dict(album_list)

In [ ]:
album_df.head()

In [ ]:
artist_df = pd.DataFrame.from_dict(artist_list)

In [ ]:
artist_df.head()

In [ ]:
song_df = pd.DataFrame.from_dict(song_list)

In [ ]:
song_df.head()

In [ ]:
song_df

In [ ]:
album_df .info()

In [ ]:
album_df['release_date'] = pd.to_datetime(album_df['release_date']) 

In [ ]:
album_df

In [ ]:
artist_df.info()

In [ ]:
artist_df

In [ ]:
song_df.info()

In [ ]:
song_df['song_added'] = pd.to_datetime(song_df['song_added']) 

In [ ]:
song_df

Save the DataFrame to a CSV file

In [ ]:
 album_df.to_csv('Albums.csv', index=False)
 artist_df.to_csv('Artists.csv', index=False)
 song_df.to_csv('Songs.csv', index=False)
 print("CSV file saved successfully!")